In [ ]:
import subprocess
filename = "/home/ande188/CIMExport.xml"
url = "http://localhost:8889/bigdata/namespace/kb/sparql"
# subprocess.call(["curl", "-s", "-D-", "-H", "Content-Type: application/xml", "--upload-file", filename, "-X", "Post", url])


In [ ]:
import importlib
cim_profile = "rc4_2021"
cim = importlib.import_module("cimgraph.data_profile." + cim_profile)

In [ ]:
from cimgraph.databases import Parameter, ConnectionParameters
from cimgraph.databases.blazegraph.blazegraph import BlazegraphConnection
# from cimgraph.databases.gridappsd import GridappsdConnection, get_topology_response
from cimgraph.models import NodeBreakerModel
import json
import time

In [ ]:
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",  cim_profile=cim_profile, 
                              iec61970_301=6,namespace="http://iec.ch/TC57/2011/CIM-schema-cim15#")
bg = BlazegraphConnection(params)

In [ ]:
geo_region_id = "_b68c7bbf-9b8e-46ee-84ce-c92793a57c9d"
geo_region = cim.GeographicalRegion(mRID = geo_region_id)

In [ ]:
network = NodeBreakerModel(connection=bg, container=geo_region, distributed=False)
network.add_to_graph(geo_region)

In [ ]:
cim_class = cim.SubGeographicalRegion

In [ ]:
network.get_all_edges(cim_class)
network.pprint(cim_class)

In [ ]:
feeder = {}
feeder["container"] = "Feeder"
feeder["contains"] = []

bay = {}
bay["container"] = "Bay"
bay["contains"] = []

vl_area = {}
vl_area["container"] = "VoltageLevel"
vl_area["contains"] = [{"feeders": feeder}, {"bays": bay}]

sub_area = {}
sub_area["container"] = "Substation"
sub_area["contains"] = [{"voltage_levels": vl_area}]

subregion = {}
subregion["container"] = "SubGeographicalRegion"
subregion["contains"] = [{"substations": sub_area}]

region = {}
region["container"] = "GeographicalRegion"
region["contains"] = [{"subregions": subregion}]

distributed_hierarchy = [{"regions": region}]

print(json.dumps(distributed_hierarchy, indent=2))

In [ ]:
feeder = {}
feeder["container"] = "Feeder"
feeder["contains"] = []

bay = {}
bay["container"] = "Bay"
bay["contains"] = []

vl_area = {}
vl_area["container"] = "VoltageLevel"
vl_area["contains"] = [feeder, bay]

sub_area = {}
sub_area["container"] = "Substation"
sub_area["contains"] = [vl_area]

subregion = {}
subregion["container"] = "SubGeographicalRegion"
subregion["contains"] = [sub_area]

region = {}
region["container"] = "GeographicalRegion"
region["contains"] = [subregion]

distributed_hierarchy = [region]

print(json.dumps(distributed_hierarchy, indent=4))

In [ ]:
network = NodeBreakerModel(connection=bg, container=geo_region, distributed=True, distributed_hierarchy=distributed_hierarchy)

In [ ]:

for sr_area in network.distributed_areas[cim.SubGeographicalRegion].values():
    print("subregion", sr_area.container.name)
    for sub_area in sr_area.distributed_areas[cim.Substation].values():
        print("substation", sub_area.container.name)
        for vl_area in sub_area.distributed_areas[cim.VoltageLevel].values():
            print("voltage level", vl_area.container.name)
            # vl_area.pprint(cim.ConnectivityNode)


In [ ]:
len(network.distributed_areas[cim.SubGeographicalRegion])

In [ ]:
feeder = {}
feeder["container"] = "Feeder"
feeder["parent"] = "Feeder.NormalEnergizingSubstation"
feeder["children"] = []

bay = {}
bay["container"] = "Bay"
bay["parent"] = "Bay.Substation"
bay["children"] = []

vl_area = {}
vl_area["container"] = "VoltageLevel"
vl_area["parent"] = "VoltageLevel.Substation"
vl_area["children"] = [feeder, bay]

sub_area = {}
sub_area["container"] = "Substation"
sub_area["parent"] = "Substation.Region"
sub_area["children"] = [vl_area]

subregion = {}
subregion["container"] = "SubGeographicalRegion"
subregion["parent"] = "SubGeographicalRegion.Region"
subregion["contains"] = [sub_area]

region = {}
region["container"] = "GeographicalRegion"
region["parent"] = ""
region["contains"] = [subregion]

distributed_hierarchy = [region]

print(json.dumps(distributed_hierarchy, indent=4))